In [ ]:
!pip install -q sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.7 MB/s eta 0:00:00


In [ ]:

file_path = "bns.txt"

with open(file_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(lines)} sections")
print("Sample:", lines[0])


Loaded 358 sections
Sample: BNS Section 1 – Short Title, Commencement & Application – Specifies the name as Bharatiya Nyaya Sanhita, its commencement date, nationwide jurisdiction including extraterritorial applicability, and its role in replacing the Indian Penal Code, 1860.


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(lines, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
import pickle

with open("bns_embeddings.pkl", "wb") as f:
    pickle.dump((lines, embeddings), f)

print("✅ Embeddings saved!")


✅ Embeddings saved!


In [ ]:
import pickle

with open("bns_embeddings.pkl", "rb") as f:
    lines, embeddings = pickle.load(f)

print(f"✅ Loaded {len(lines)} legal entries")


✅ Loaded 358 legal entries


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
def search_bns(query, top_k=3):
    query_embedding = model.encode([query])
    scores = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = np.argsort(scores)[-top_k:][::-1]

    results = [(lines[i], scores[i]) for i in top_indices]
    return results


In [ ]:
query = "What is the punishment for theft in a home?"
results = search_bns(query)

print("📘 Top matches:\n")
for res, score in results:
    print(f"→ {res}\n(similarity: {score:.2f})\n")


📘 Top matches:

→ BNS Section 303 – Theft – Defines theft as dishonest removal of movable property from another person’s possession without consent.
(similarity: 0.55)

→ BNS Section 305 – Theft in Dwelling House, Vessel, or Enclosed Place – Provides higher punishment when theft occurs inside a house, ship, or secure premises.
(similarity: 0.55)

→ BNS Section 319 – Receiving Property Stolen in Robbery or Dacoity – Punishes those who knowingly accept goods acquired through robbery or dacoity.
(similarity: 0.51)



In [ ]:
query = "What is section 302?"
results = search_bns(query)

print("📘 Top matches:\n")
for res, score in results:
    print(f"→ {res}\n(similarity: {score:.2f})\n")

📘 Top matches:

→ BNS Section 302 – Attempt to Commit Criminal Intimidation – Applies when a person takes steps to instill fear or coerce another without fulfilling the act.
(similarity: 0.50)

→ BNS Section 178 – Giving False Information – Addresses knowingly providing false details to a public servant, especially during official duties or inquiries.
(similarity: 0.47)

→ BNS Section 187 – Providing False Information to Mislead Officer – Addresses giving misleading or incorrect information to a police officer to derail investigation or implicate another.
(similarity: 0.46)



In [ ]:
!pip install openai --quiet


In [ ]:
import os, openai

openai.api_key = "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"
openai.api_base = "https://api.groq.com/openai/v1"

model = "gemma2-9b-it"

response = openai.ChatCompletion.create(
    model="gemma2-9b-it",
    messages=[
        {"role": "system", "content": "You are a helpful legal assistant for Indian law. Only refer to the Bharatiya Nyaya Sanhita (BNS) 2023, not any other country's laws."},
        {"role": "user", "content": "Explain BNS Section 305: Theft in Dwelling House, Vessel, or Enclosed Place."}
    ]
)


print(response.choices[0].message.content)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
